Intro configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.integrate import simps
import os
import pickle
import csv

In [ ]:
% cd drive/My Drive/Datasets/Iteration_1/EEG_epochs_csv
% ls

In [ ]:
# constants for managing files
channels_names = ['FP1', 'FP2', 'F7', 'F3', 'FZ', 'F4', 'F8', 'FT7', 'FC3', 'FCZ', 'FC4',
       'FT8', 'T3', 'C3', 'CZ', 'C4', 'T4', 'TP7', 'CP3', 'CPZ', 'CP4', 'TP8',
       'T5', 'P3', 'PZ', 'P4', 'T6', 'O1', 'OZ', 'O2']
input_file_column_names = ['FP1', 'FP2', 'F7', 'F3', 'FZ', 'F4', 'F8', 'FT7', 'FC3', 'FCZ', 'FC4',
       'FT8', 'T3', 'C3', 'CZ', 'C4', 'T4', 'TP7', 'CP3', 'CPZ', 'CP4', 'TP8',
       'T5', 'P3', 'PZ', 'P4', 'T6', 'O1', 'OZ', 'O2', 'Time']
events_numbers = {}
new_column_names = None
bandpowers_dir_name = "Bandpowers"
timerelated_dir_name = "Time_related"
bandpowers_dir = os.path.join(os.getcwd(), bandpowers_dir_name)
timerelated_dir = os.path.join(os.getcwd(), timerelated_dir_name)                              
filenames_time_related = [f for f in os.listdir(timerelated_dir) if os.path.isfile(os.path.join(timerelated_dir, f))]
filenames_bandpowers = [f for f in os.listdir(bandpowers_dir) if os.path.isfile(os.path.join(bandpowers_dir, f))]

In [ ]:
# constants for signal processing
sampling_freq = 500.

delta_min = 0.01
delta_max = 4
theta_min = 4
theta_max = 7
alpha_min = 8
alpha_max = 12
beta_min = 12
beta_max = 30
gamma_min = 30

delta = (delta_min, delta_max)
theta = (theta_min, theta_max)
alpha = (alpha_min, alpha_max)
beta = (beta_min, beta_max)

bands_ranges = [delta, theta, alpha, beta, gamma_min]
bands_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']
bands = dict(zip(bands_names, bands_ranges))

# Helper functions

In [ ]:
def assert_file_numbers(filenames_list, number=34):
  try:
    assert(len(filenames_list) == number)
  except:
    print(f"Filenames list: {len(filenames_list)}. Expected: {number}")
    raise AssertionError

In [ ]:
def assert_correct_file(df, filename):
  input_cols = list(df)
  name = filename.split(" ", 1)[0]

  assert input_cols.sort() == input_file_column_names.sort()
  assert events_numbers.get(name)
  assert len(df) == events_numbers[name]


In [ ]:
def initialize_constants(df):
  channels = df.columns
  channels = channels.drop("Time")

  new_columns = []
  for c in channels:
    for b in bands_names:
      new_columns.append(c + "_" + b)

  return channels, new_columns

In [ ]:
def bandpower(data, sf, band, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps

    # Compute the modified periodogram (Welch)
    freqs, psd = welch(data, sf)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    band = np.asarray(band)
    if band.size == 2:
      low, high = band
    else:
      low = band
      high = freqs.max()

    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [ ]:
def get_chunk_from_df(dataframe, chunk_size=1500):
  """Get next epoch records from dataframe

    Parameters
    ----------
    dataframe : pandas.DataFrame
        From eeg file exported from Matlab
    chunk_size : int
        Number of dataframe rows corresponding with one epoch. Always 1500 for this dataset

    Return
    ------
    generator : pandas.DataFrame
        With next 1500 rows
    """
  for start_row in range(0, dataframe.shape[0], chunk_size):
      end_row  = min(start_row + chunk_size, dataframe.shape[0])
      yield dataframe.iloc[start_row:end_row, :]

In [ ]:
def process_eeg_time_to_bandpower(df):
  # initialize generator to fetching epochs from eeg dataframe
  get_epoch = get_chunk_from_df(df)
  list_eeg_proc = []
  epochs_number = df[df["Time"] == -3000].shape[0]
  print(f"Control: {epochs_number} events")
  for epoch_num in range(epochs_number):
    # fetch next epoch
    epoch = next(get_epoch)
    epoch_data = []

    # process channels for this epoch
    for channel_name in channels_names:
      epoch_channel = epoch[channel_name]
      channel_bandpowers = []

      # compute bandpowers for each of bands for this channel
      for band_name, bandwidth in bands.items():
        bp = bandpower(epoch_channel, sampling_freq, bandwidth, relative=True)
        channel_bandpowers.append(bp)
      epoch_data += channel_bandpowers
    list_eeg_proc.append(epoch_data)

  return pd.DataFrame(data=list_eeg_proc, columns=new_column_names)

# EEG Processing

In [ ]:
filenames_time_related.sort()
filenames_time_related

In [ ]:
assert_file_numbers(filenames_time_related, number=14)
events_numbers = {}
# process each time-related epochs file from matlab to bandpowers file
for f in filenames_time_related:
  df = pd.read_csv(os.path.join(timerelated_dir, f), sep=',')
  epochs_number = df[df["Time"] == -3000].shape[0]
  
  if new_column_names is None:
    channels_names, new_column_names = initialize_constants(df)
    print(new_column_names)

  print(f"Processing {f}")
  # assert_correct_file(df, f)
  df_proc = process_eeg_time_to_bandpower(df)

  name, ext = os.path.splitext(f)
  new_name = name + "_bandpower"
  events_numbers[name] = epochs_number
  df_proc.to_csv(f"{bandpowers_dir_name}/{new_name}{ext}")
  print("Saved")

['FP1_delta', 'FP1_theta', 'FP1_alpha', 'FP1_beta', 'FP1_gamma', 'FP2_delta', 'FP2_theta', 'FP2_alpha', 'FP2_beta', 'FP2_gamma', 'F7_delta', 'F7_theta', 'F7_alpha', 'F7_beta', 'F7_gamma', 'F3_delta', 'F3_theta', 'F3_alpha', 'F3_beta', 'F3_gamma', 'FZ_delta', 'FZ_theta', 'FZ_alpha', 'FZ_beta', 'FZ_gamma', 'F4_delta', 'F4_theta', 'F4_alpha', 'F4_beta', 'F4_gamma', 'F8_delta', 'F8_theta', 'F8_alpha', 'F8_beta', 'F8_gamma', 'FT7_delta', 'FT7_theta', 'FT7_alpha', 'FT7_beta', 'FT7_gamma', 'FC3_delta', 'FC3_theta', 'FC3_alpha', 'FC3_beta', 'FC3_gamma', 'FCZ_delta', 'FCZ_theta', 'FCZ_alpha', 'FCZ_beta', 'FCZ_gamma', 'FC4_delta', 'FC4_theta', 'FC4_alpha', 'FC4_beta', 'FC4_gamma', 'FT8_delta', 'FT8_theta', 'FT8_alpha', 'FT8_beta', 'FT8_gamma', 'T3_delta', 'T3_theta', 'T3_alpha', 'T3_beta', 'T3_gamma', 'C3_delta', 'C3_theta', 'C3_alpha', 'C3_beta', 'C3_gamma', 'CZ_delta', 'CZ_theta', 'CZ_alpha', 'CZ_beta', 'CZ_gamma', 'C4_delta', 'C4_theta', 'C4_alpha', 'C4_beta', 'C4_gamma', 'T4_delta', 'T4_thet

In [ ]:
# combine all bandpower files
filenames_bandpowers = [f for f in os.listdir(bandpowers_dir) if os.path.isfile(os.path.join(bandpowers_dir, f))]
assert_file_numbers(filenames_bandpowers, number=28)
df_list = []
for f in filenames_bandpowers:
  df = pd.read_csv(os.path.join(bandpowers_dir, f))
  # assert_correct_file(df, f)
  print(df.shape)
  df_list.append(df)

df_eeg = pd.concat(df_list)
df_eeg

(343, 152)
(262, 152)
(322, 152)
(439, 152)
(333, 152)
(333, 152)
(269, 152)
(272, 152)
(430, 152)
(188, 152)
(359, 152)
(361, 152)
(332, 152)
(199, 152)
(199, 151)
(332, 151)
(361, 151)
(359, 151)
(188, 151)
(430, 151)
(272, 151)
(269, 151)
(333, 151)
(333, 151)
(439, 151)
(322, 151)
(262, 151)
(343, 151)


,Unnamed: 0,FP1_delta,FP1_theta,FP1_alpha,FP1_beta,FP1_gamma,FP2_delta,FP2_theta,FP2_alpha,FP2_beta,FP2_gamma,F7_delta,F7_theta,F7_alpha,F7_beta,F7_gamma,F3_delta,F3_theta,F3_alpha,F3_beta,F3_gamma,FZ_delta,FZ_theta,FZ_alpha,FZ_beta,FZ_gamma,F4_delta,F4_theta,F4_alpha,F4_beta,F4_gamma,F8_delta,F8_theta,F8_alpha,F8_beta,F8_gamma,FT7_delta,FT7_theta,FT7_alpha,FT7_beta,...,T5_theta,T5_alpha,T5_beta,T5_gamma,P3_delta,P3_theta,P3_alpha,P3_beta,P3_gamma,PZ_delta,PZ_theta,PZ_alpha,PZ_beta,PZ_gamma,P4_delta,P4_theta,P4_alpha,P4_beta,P4_gamma,T6_delta,T6_theta,T6_alpha,T6_beta,T6_gamma,O1_delta,O1_theta,O1_alpha,O1_beta,O1_gamma,OZ_delta,OZ_theta,OZ_alpha,OZ_beta,OZ_gamma,O2_delta,O2_theta,O2_alpha,O2_beta,O2_gamma,label
0,0,0.079218,0.0,0.017556,0.308258,0.284100,0.022650,0.0,0.010188,0.410456,0.450478,0.050500,0.0,0.044850,0.331536,0.279191,0.101590,0.0,0.041662,0.256914,0.212431,0.154676,0.0,0.032200,0.210368,0.098388,0.027445,0.0,0.019565,0.380428,0.428065,0.085909,0.0,0.029635,0.269359,0.269014,0.043222,0.0,0.031994,0.356799,...,0.0,0.037101,0.330639,0.288118,0.183850,0.0,0.039835,0.225251,0.108167,0.224602,0.0,0.035348,0.169133,0.081354,0.202612,0.0,0.038136,0.194742,0.098970,0.096987,0.0,0.035727,0.335589,0.207593,0.110636,0.0,0.037329,0.286941,0.248949,0.139743,0.0,0.038232,0.264140,0.202411,0.154352,0.0,0.036247,0.258749,0.176217,attentive
1,1,0.089648,0.0,0.033712,0.365459,0.249493,0.027317,0.0,0.010104,0.427481,0.387499,0.106005,0.0,0.018726,0.340662,0.289788,0.229237,0.0,0.027405,0.179961,0.151896,0.277509,0.0,0.036320,0.104902,0.070545,0.108187,0.0,0.012696,0.293268,0.316280,0.125466,0.0,0.013871,0.262780,0.302425,0.212783,0.0,0.023905,0.250575,...,0.0,0.027683,0.344047,0.187743,0.274458,0.0,0.033616,0.139426,0.068173,0.291722,0.0,0.030253,0.102284,0.050509,0.279125,0.0,0.024219,0.118652,0.073785,0.154348,0.0,0.024315,0.301737,0.189242,0.147109,0.0,0.048197,0.274822,0.220195,0.179082,0.0,0.040017,0.235277,0.170229,0.174592,0.0,0.032411,0.234552,0.159578,NaN
2,2,0.120319,0.0,0.020572,0.321629,0.193533,0.040478,0.0,0.008063,0.427453,0.329595,0.067476,0.0,0.014486,0.433005,0.283749,0.117663,0.0,0.026158,0.320155,0.190391,0.209174,0.0,0.031554,0.159849,0.109850,0.036796,0.0,0.010517,0.410663,0.319085,0.023853,0.0,0.020028,0.388869,0.391659,0.062083,0.0,0.036260,0.313100,...,0.0,0.027292,0.389838,0.227894,0.259426,0.0,0.024521,0.152607,0.088536,0.282588,0.0,0.028469,0.111618,0.053393,0.267557,0.0,0.034495,0.119387,0.069721,0.164880,0.0,0.044407,0.252377,0.148233,0.195664,0.0,0.028775,0.213315,0.167891,0.225116,0.0,0.031803,0.182185,0.115223,0.218419,0.0,0.042076,0.185819,0.100303,NaN
3,3,0.082108,0.0,0.036865,0.327361,0.278492,0.015819,0.0,0.022506,0.506953,0.312016,0.041385,0.0,0.049939,0.326721,0.332768,0.106941,0.0,0.042572,0.353514,0.188432,0.144232,0.0,0.054260,0.252095,0.113427,0.034683,0.0,0.010337,0.522957,0.261670,0.045489,0.0,0.046865,0.337942,0.340486,0.056624,0.0,0.064954,0.382316,...,0.0,0.040699,0.371561,0.264492,0.153259,0.0,0.088624,0.229957,0.116604,0.179910,0.0,0.079678,0.216415,0.088377,0.170531,0.0,0.065023,0.232684,0.108218,0.190458,0.0,0.067017,0.192293,0.111265,0.093530,0.0,0.051275,0.274916,0.254782,0.084709,0.0,0.053276,0.302612,0.236363,0.082951,0.0,0.049345,0.319696,0.221454,attentive
4,4,0.061539,0.0,0.018902,0.481675,0.189174,0.014003,0.0,0.006487,0.341193,0.433712,0.040511,0.0,0.045510,0.358302,0.272069,0.100055,0.0,0.047163,0.324084,0.142606,0.142394,0.0,0.074186,0.169012,0.094992,0.018851,0.0,0.023366,0.410753,0.351194,0.037373,0.0,0.023475,0.350946,0.411341,0.101969,0.0,0.054870,0.303642,...,0.0,0.051548,0.410245,0.251719,0.130050,0.0,0.072852,0.264896,0.116825,0.151016,0.0,0.057263,0.241543,0.111621,0.137602,0.0,0.066392,0.241220,0.140073,0.078155,0.0,0.028362,0.453416,0.205163,0.055165,0.0,0.042590,0.414011,0.275024,0.072760,0.0,0.040379,0.416733,0.233510,0.080792,0.0,0.042963,0.420452,0.207969,attentive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Prepare data for neural network

In [ ]:
df_eeg_proc = df_eeg[df_eeg['label'].notna()]
df_eeg_proc = df_eeg_proc.drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
df_eeg_proc

,FP1_delta,FP1_theta,FP1_alpha,FP1_beta,FP1_gamma,FP2_delta,FP2_theta,FP2_alpha,FP2_beta,FP2_gamma,F7_delta,F7_theta,F7_alpha,F7_beta,F7_gamma,F3_delta,F3_theta,F3_alpha,F3_beta,F3_gamma,FZ_delta,FZ_theta,FZ_alpha,FZ_beta,FZ_gamma,F4_delta,F4_theta,F4_alpha,F4_beta,F4_gamma,F8_delta,F8_theta,F8_alpha,F8_beta,F8_gamma,FT7_delta,FT7_theta,FT7_alpha,FT7_beta,FT7_gamma,...,T5_theta,T5_alpha,T5_beta,T5_gamma,P3_delta,P3_theta,P3_alpha,P3_beta,P3_gamma,PZ_delta,PZ_theta,PZ_alpha,PZ_beta,PZ_gamma,P4_delta,P4_theta,P4_alpha,P4_beta,P4_gamma,T6_delta,T6_theta,T6_alpha,T6_beta,T6_gamma,O1_delta,O1_theta,O1_alpha,O1_beta,O1_gamma,OZ_delta,OZ_theta,OZ_alpha,OZ_beta,OZ_gamma,O2_delta,O2_theta,O2_alpha,O2_beta,O2_gamma,label
0,0.079218,0.0,0.017556,0.308258,0.284100,0.022650,0.0,0.010188,0.410456,0.450478,0.050500,0.0,0.044850,0.331536,0.279191,0.101590,0.0,0.041662,0.256914,0.212431,0.154676,0.0,0.032200,0.210368,0.098388,0.027445,0.0,0.019565,0.380428,0.428065,0.085909,0.0,0.029635,0.269359,0.269014,0.043222,0.0,0.031994,0.356799,0.288684,...,0.0,0.037101,0.330639,0.288118,0.183850,0.0,0.039835,0.225251,0.108167,0.224602,0.0,0.035348,0.169133,0.081354,0.202612,0.0,0.038136,0.194742,0.098970,0.096987,0.0,0.035727,0.335589,0.207593,0.110636,0.0,0.037329,0.286941,0.248949,0.139743,0.0,0.038232,0.264140,0.202411,0.154352,0.0,0.036247,0.258749,0.176217,attentive
1,0.082108,0.0,0.036865,0.327361,0.278492,0.015819,0.0,0.022506,0.506953,0.312016,0.041385,0.0,0.049939,0.326721,0.332768,0.106941,0.0,0.042572,0.353514,0.188432,0.144232,0.0,0.054260,0.252095,0.113427,0.034683,0.0,0.010337,0.522957,0.261670,0.045489,0.0,0.046865,0.337942,0.340486,0.056624,0.0,0.064954,0.382316,0.242920,...,0.0,0.040699,0.371561,0.264492,0.153259,0.0,0.088624,0.229957,0.116604,0.179910,0.0,0.079678,0.216415,0.088377,0.170531,0.0,0.065023,0.232684,0.108218,0.190458,0.0,0.067017,0.192293,0.111265,0.093530,0.0,0.051275,0.274916,0.254782,0.084709,0.0,0.053276,0.302612,0.236363,0.082951,0.0,0.049345,0.319696,0.221454,attentive
2,0.061539,0.0,0.018902,0.481675,0.189174,0.014003,0.0,0.006487,0.341193,0.433712,0.040511,0.0,0.045510,0.358302,0.272069,0.100055,0.0,0.047163,0.324084,0.142606,0.142394,0.0,0.074186,0.169012,0.094992,0.018851,0.0,0.023366,0.410753,0.351194,0.037373,0.0,0.023475,0.350946,0.411341,0.101969,0.0,0.054870,0.303642,0.235090,...,0.0,0.051548,0.410245,0.251719,0.130050,0.0,0.072852,0.264896,0.116825,0.151016,0.0,0.057263,0.241543,0.111621,0.137602,0.0,0.066392,0.241220,0.140073,0.078155,0.0,0.028362,0.453416,0.205163,0.055165,0.0,0.042590,0.414011,0.275024,0.072760,0.0,0.040379,0.416733,0.233510,0.080792,0.0,0.042963,0.420452,0.207969,attentive
3,0.065643,0.0,0.020158,0.410403,0.236055,0.014123,0.0,0.010175,0.444961,0.390263,0.052167,0.0,0.017807,0.402665,0.324085,0.122571,0.0,0.021978,0.377021,0.171640,0.192163,0.0,0.024860,0.232837,0.110163,0.041780,0.0,0.012972,0.466956,0.302499,0.037682,0.0,0.026883,0.465461,0.274161,0.076837,0.0,0.022402,0.370780,0.267457,...,0.0,0.031583,0.400240,0.272825,0.156748,0.0,0.038578,0.262178,0.106259,0.195851,0.0,0.045119,0.204020,0.078917,0.183843,0.0,0.041801,0.253137,0.085127,0.098852,0.0,0.065199,0.357531,0.203072,0.098325,0.0,0.037223,0.358845,0.201902,0.138834,0.0,0.039241,0.304779,0.155307,0.161336,0.0,0.040817,0.276337,0.139780,attentive
4,0.055973,0.0,0.016529,0.291960,0.372223,0.019859,0.0,0.007404,0.362036,0.400946,0.030239,0.0,0.026688,0.332285,0.396825,0.071398,0.0,0.019051,0.340719,0.242631,0.127536,0.0,0.026477,0.192099,0.113072,0.033801,0.0,0.009210,0.280014,0.460445,0.060451,0.0,0.017447,0.274202,0.406494,0.085969,0.0,0.016959,0.264980,0.354079,...,0.0,0.016583,0.405807,0.198405,0.153423,0.0,0.032645,0.226117,0.143970,0.167870,0.0,0.037622,0.207004,0.110308,0.144537,0.0,0.035653,0.246600,0.135223,0.088146,0.0,0.041374,0.332156,0.239987,0.068024,0.0,0.038107,0.312468,0.268353,0.094698,0.0,0.039251,0.272025,0.236283,0.124715,0.0,0.038554,0.229103,0.218988,attentive
...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
df_attentive = df_eeg_proc[df_eeg_proc['label']=='attentive'].drop(['label'], axis=1)
df_inattentive = df_eeg_proc[df_eeg_proc['label']=='inattentive'].drop(['label'], axis=1)

In [ ]:
# reformat dataset from 2D dataframes to 3D tensors
chunks_list = []
labels_list = []
chunk_size = 8
get_chunk = get_chunk_from_df(df_attentive, chunk_size=chunk_size)
for chunk_num in range(int(df_attentive.shape[0]/chunk_size)):
    chunk = next(get_chunk)
    new_chunk = chunk.to_numpy()
    chunks_list.append(new_chunk)
    labels_list.append('attentive')

print(f"Attentive chunks list: {len(labels_list)}")

get_chunk = get_chunk_from_df(df_inattentive, chunk_size=chunk_size)
for chunk_num in range(int(df_inattentive.shape[0]/chunk_size)):
    chunk = next(get_chunk)
    new_chunk = chunk.to_numpy()
    chunks_list.append(new_chunk)
    labels_list.append('inattentive')

arr_eegnet = np.asarray(chunks_list)
arr_eegnet = np.swapaxes(arr_eegnet, 1, 2)
print(f"Output array shape: {arr_eegnet.shape}")
print(f"Labels list: {len(labels_list)}")

Attentive chunks list: 200
Output array shape: (303, 150, 8)
Labels list: 303


In [ ]:
# varbose to numeric labels
labels_mapping = {"attentive": 0, "inattentive": 1}
labels_2class = [labels_mapping[label] for label in labels_list]
labels_2class = np.asarray(labels_2class)

In [ ]:
# save dataset
pd.DataFrame(data=labels_2class).to_csv("y_DD_features.csv")
pickle.dump(arr_eegnet, open("X_DD_features.pkl", "wb"))